In [1]:
import os
import json
import importlib

import glob as gb
import utils as ut
import numpy as np
import pandas as pd
import seaborn as sns
import ipywidgets as pyw
import matplotlib.pyplot as plt

from IPython.display import clear_output

# Global

In [2]:
%matplotlib inline

In [3]:
importlib.reload(ut)

plt.rcParams['text.usetex'] = True
plt.style.use(['dark_background'])

# Density

 ### $\tilde{D}$ ... integrated density

$$
\begin{aligned}
    \tilde{D} &= 1 - \left(1 - D\right)^{\frac{l}{o \cdot \cos\left(\alpha\right)}} \qquad \qquad 
    \begin{aligned}
        &\text{D} &&\text{... uniform density per slice} \\
        &\text{l} &&\text{... occluder volume height} \\
        &\text{o} &&\text{... occluder cubic size} \\
        &\alpha &&\text{... obligue angle}, \frac{FOV}{2}
    \end{aligned}
\end{aligned}
$$

In [4]:
def D_tilde(D, lo, a):
    return 1 - (1 - D)**(lo * (1 / np.cos(np.deg2rad(a))))

In [5]:
@pyw.interact
def plot(lo=pyw.IntRangeSlider(value=(0, 300), min=0, max=300, continuous_update=True),
         a=pyw.IntSlider(value=0, min=0, max=90, continuous_update=True)):
    
    D = np.array([0.10, 0.25, 0.50, 1.00, 2.50, 10.00]) / 100
    D_tilde_lo_vs_D = [np.hstack([[x], D_tilde(D, x, a)]) for x in range(*lo)]
    
    # dataframe
    columns = ['lo'] + [f'D = {x*100:.2f} \%' for x in D]
    df = pd.DataFrame(D_tilde_lo_vs_D, columns=columns).set_index('lo')
    
    # plot
    ax = df.plot(figsize=(16, 8))
    ax.set(ylabel=r'density $\tilde{D}$')

interactive(children=(IntRangeSlider(value=(0, 300), description='lo', max=300), IntSlider(value=0, descriptio…

 ### $\tilde{D}_{\alpha}$ ... integrated obligue angle density

$$
\begin{aligned}
    \tilde{D}_{\alpha} &= 1 - \left(1 - \tilde{D}\right)^{\frac{1}{\cos\left(\alpha\right)}} \qquad \qquad 
    \begin{aligned}
        &\tilde{D} &&\text{... integrated density} \\
        &\alpha &&\text{... obligue angle}, \frac{FOV}{2}
    \end{aligned}
\end{aligned}
$$

In [6]:
def D_tilde_alpha(D, lo, a):
    return 1 - (1 - D_tilde(D, lo, 0))**(1 / np.cos(np.deg2rad(a)))

In [7]:
@pyw.interact
def plot(fov=pyw.IntRangeSlider(value=(0, 180), min=0, max=180, continuous_update=True),
         lo=pyw.IntSlider(value=140, min=0, max=300, continuous_update=True)):
    
    D = np.array([0.10, 0.25, 0.50, 1.00, 2.50, 10.00]) / 100
    D_tilde_fov_vs_D = [np.hstack([[x / 2], D_tilde_alpha(D, lo, x / 2)]) for x in range(*fov)]
    
    # dataframe
    columns = [r'$\alpha$ (deg)'] + [f'D = {x*100:.2f} \%' for x in D]
    df = pd.DataFrame(D_tilde_fov_vs_D, columns=columns).set_index(r'$\alpha$ (deg)')
    
    # plot
    ax = df.plot(figsize=(16, 8))
    ax.set(ylabel=r'density $\tilde{D}_{\alpha}$')

interactive(children=(IntRangeSlider(value=(0, 180), description='fov', max=180), IntSlider(value=140, descrip…

# Visibility

### $\hat{d}$ ... optimal disparity

$$
\begin{aligned}
    \hat{d} &= \frac{d}{o} \geq 1 \qquad \qquad 
    \begin{aligned}
        &\text{d} &&\text{... disparity shift between SAI samples} \\
        &\text{o} &&\text{... occluder cubic size} \\
    \end{aligned}
\end{aligned}
$$

### $V$ ... visibility assuming optimal disparity

$$
\begin{aligned}
    V &= 1 - \tilde{D}^{2} - \frac{\tilde{D}(1 - \tilde{D})}{N} \qquad \qquad 
    \begin{aligned}
        &\tilde{D} &&\text{... integrated density} \\
        &N &&\text{... number of SAI samples}
    \end{aligned}
\end{aligned}
$$

In [8]:
def V(D, N):
    return 1 - D**2 - D * (1 - D) / N

In [9]:
@pyw.interact
def plot():
    
    D = np.linspace(0, 1, 1000)
    N = np.array([1, 2, 4, 9, 25, 100])
    V_D_vs_N = [np.hstack([[x], V(x, N)]) for x in D]
    
    # dataframe
    columns = [r'density $\tilde{D}$'] + [f'N = {x}' for x in N]
    df = pd.DataFrame(V_D_vs_N, columns=columns).set_index(r'density $\tilde{D}$')
    
    # plot
    ax = df.plot(figsize=(16, 8))
    ax.set(ylabel=r'visibility $V$')

interactive(children=(Output(),), _dom_classes=('widget-interact',))

# Optimization

#### Coverage - Distance - Sampling
$$
\begin{aligned}
    I_c = 2 \cdot h \cdot \tan\left(\frac{\text{FOV}}{2}\right), \qquad \qquad I_d = v \cdot t_p , \qquad \qquad I_s = \frac{I_c}{s_d} = 30 \\
\end{aligned}
$$

#### Resolution - Overlap - Time
$$
\begin{aligned}
    I_r = \frac{I_c}{r}, \qquad \qquad I_o = \frac{I_c}{I_d}, \qquad \qquad I_t = \frac{I_c}{v} = \frac{s_d}{v} \cdot I_s
\end{aligned}
$$

## Constraints - Dynamic
$$
\begin{aligned}
    C_d &= \left\{ v, h, \text{FOV}, s_d \right\} \\
\end{aligned}
$$

## Constraints - Hardware
$$
\begin{aligned}
    C_h &= \left\{ t_p, r, f \right\} \\
\end{aligned}
$$

## Constraints - Quality
$$
\begin{aligned}
    C_q &= \left\{ I_r, I_o, I_t \right\} \\
    &= \left\{ \frac{I_c}{r}, \frac{I_c}{I_d}, \frac{I_c}{v} \right\} \\
    &= \left\{ \frac{I_c}{r}, \frac{I_c}{v \cdot t_p}, \frac{I_c}{v} \right\} \\
\end{aligned}
$$

In [10]:
@pyw.interact
def plot(fov=pyw.IntRangeSlider(value=(10, 160), min=10, max=160, continuous_update=True),
         h=pyw.IntSlider(value=35, min=1, max=100, continuous_update=True),
         v=pyw.IntSlider(value=4, min=1, max=20, continuous_update=True)):
    
    n = 10000
    
    FOV = np.linspace(*fov, n)
    H = np.repeat(h, n)
    V = np.repeat(v, n)
    
    s_d = 1
    
    # constraints - hardware
    t_p = 0.5
    r = 512
    f = 10
    
    # coverage - distance - sampling
    I_c = 2 * H * np.tan(np.deg2rad(FOV / 2))
    I_d = V * t_p
    I_s = I_c / s_d # = 30
    
    # resolution - overlap - time
    I_r = I_c / r
    I_o = I_c / I_d
    I_t = I_c / V
    
    # constraints
    C_h = (t_p, r, f)
    C_d = (V, H, FOV, s_d)
    C_b = (I_c, I_d, I_s)
    C_q = (I_r, I_o, I_t)
    
    # dataframe
    columns = [r'FOV'] + [r'Coverage ($I_c$)', r'Resolution ($I_r$)', r'Overlap ($I_o$)', r'Time ($I_t$)']
    df = pd.DataFrame(np.vstack([[FOV], [I_c], C_q]).T, columns=columns).set_index(r'FOV')
    
    # plot
    ax = df.plot(figsize=(16, 8))
    ax.set(ylabel=r'quality $C_q$')

interactive(children=(IntRangeSlider(value=(10, 160), description='fov', max=160, min=10), IntSlider(value=35,…